In [1]:
! sudo apt update -y && sudo apt-get install -y poppler-utils -qq

Hit:1 http://deb.debian.org/debian bullseye InRelease  
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]        
Hit:4 https://dl.yarnpkg.com/debian stable InRelease   
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [245 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages.diff/Index [20.7 kB]
Get:7 http://deb.debian.org/debian bullseye-updates/main amd64 Packages T-2023-08-26-1408.20-F-2023-08-26-1408.20.pdiff [464 B]
Get:7 http://deb.debian.org/debian bullseye-updates/main amd64 Packages T-2023-08-26-1408.20-F-2023-08-26-1408.20.pdiff [464 B]
Fetched 358 kB in 2s (232 kB/s)33m
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
6 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [2]:
! pip install pdf2image fpdf2 datasets --quiet


In [5]:
import datasets
from pdf2image import convert_from_bytes
from fpdf import FPDF
from io import BytesIO
from PIL import Image
import os
import json
import pandas as pd
SELECT_RANGE = 10
DATASET = "cc_news"

In [7]:
class SimpleDoc(FPDF):
    def __init__(self) -> None:
        super().__init__()
        self.ch = 8
        return

    def header(self) -> None:
        return

    def footer(self) -> None:
        return
    
    def body(self, text: str) -> None:
        ch = self.ch
        text = text.encode("UTF-8")
        self.add_page()
        self.set_font('Arial', 'B', 8)
        self.multi_cell(0, 5, text.decode("Latin-1"))
        # Line break
        self.ln()

In [8]:
doc_set = (
    datasets.load_dataset(DATASET)["train"]
    .shuffle()
    .select(range(SELECT_RANGE))
    .select_columns(["text"])
).to_pandas()


In [9]:
metadata = []
for i, row in enumerate(doc_set.itertuples()):
    doc = SimpleDoc()
    doc.body(text=row.text)
    stream = BytesIO(doc.output(dest="S"))
    stream.seek(0)
    base_dir = "/tmp/cc_news_images"
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    for j, image in enumerate(convert_from_bytes(stream.read(), 200)):
        filename = f"{base_dir}/doc_{i}-{j}.png"
        record = dict(file_name=f"doc_{i}-{j}.png", text=row.text)
        metadata.append(record)
        image.save(filename)
    del(doc)
    del(stream)

/tmp/ipykernel_401735/233455760.py:17: UserWarning: Substituting font arial by core font helvetica
  self.set_font('Arial', 'B', 8)
/tmp/ipykernel_401735/4256289548.py:5: DeprecationWarning: "dest" parameter is deprecated, unused and will soon be removed
  stream = BytesIO(doc.output(dest="S"))
/tmp/ipykernel_401735/233455760.py:17: UserWarning: Substituting font arial by core font helvetica
  self.set_font('Arial', 'B', 8)
/tmp/ipykernel_401735/4256289548.py:5: DeprecationWarning: "dest" parameter is deprecated, unused and will soon be removed
  stream = BytesIO(doc.output(dest="S"))
/tmp/ipykernel_401735/233455760.py:17: UserWarning: Substituting font arial by core font helvetica
  self.set_font('Arial', 'B', 8)
/tmp/ipykernel_401735/4256289548.py:5: DeprecationWarning: "dest" parameter is deprecated, unused and will soon be removed
  stream = BytesIO(doc.output(dest="S"))
/tmp/ipykernel_401735/233455760.py:17: UserWarning: Substituting font arial by core font helvetica
  self.set_fo

In [10]:
pd.DataFrame(metadata).to_csv(f"{base_dir}/metadata.csv", index=False)

In [13]:
dt_sample = datasets.load_dataset("imagefolder", data_dir="/tmp/cc_news_images/", split="train[:20%]")
dt_train = datasets.load_dataset("imagefolder", data_dir="/tmp/cc_news_images/", split="train[:70%]")
dt_test = datasets.load_dataset("imagefolder", data_dir="/tmp/cc_news_images/", split="train[70%:]")

Resolving data files: 100%|██████████| 258/258 [00:00<00:00, 573162.31it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 2 examples [00:00, 599.40 examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [14]:
tbl = datasets.DatasetDict(
    {
        "sample": dt_sample,
        "train": dt_train,
        "test": dt_test, 
    }
)
tbl

NameError: name 'dt_sample' is not defined

In [12]:
# dt.push_to_hub("lowem1/cc_news_images")